# QB Stat Predictor

## Intro

<p>
Have got a large dataset from profootbal reference which has the stats from all games for almost every player <br>
I have taken Aaron Rodgers stats over every game <br>
The aim is to see if I can accurately predict upcoming stats. <br>
So model will be trained on data from beginning of player career. <br>
Then will be tested on this seasons (2025) games to see how it performs. 
</p>

<p>
I feel for the dataset I need to define all the column names for my own sake<br>
I'll only do the technical ones - some are self explanatory
<br><br>
Yds         = yards gained by passing
<br><br>
Y/A         = yards gained per pass attempt
<br><br>
AY/A        = adjusted yards per pass attempt - To give a more comprehensive view of a quarterback's    performance than a simple yards-per-attempt average by including the impact of touchdowns and interceptions. Touchdowns are weighted positively, while interceptions are weighted negatively, providing a more robust single-number metric for evaluating a quarterback. 
<br><br>
Rate        = passer rating
<br><br>
Sk          = sacks
<br><br>
Yds         = yards LOST due to sacking (change this to yds_sk)
<br><br>
Att         = rushing attempts (change to att_rush)
<br><br>
Yds         = total rushing yards (change to yds_rush)
<br><br>
TD          = rushing TD (change to td_rush)
<br><br>
Y/A         = rushing yards per attempt (change to Y/A_rush)
<br><br>

</p>

## Import Libraries

In [ ]:
#import necessary libraries

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Import Datasets

In [ ]:
#define path to datasets

#regular season
path_to_dataset = r'C:\Users\ronan\Downloads\Datasets\NFLdata\Aaron_rodgers.xlsx'
df_reg = pd.read_excel(path_to_dataset, header=1)

#playoffs
path_to_dataset = r'C:\Users\ronan\Downloads\Datasets\NFLdata\aaron_rodgers_poff.xlsx'
df_post = pd.read_excel(path_to_dataset, header=1)

In [ ]:
df_reg.head()

## Prep the dataframes

In [ ]:
#make column to say whether its a regular or post season game

#regular season
df_reg['phase'] = 'reg'
col_pos = df_reg.columns.get_loc('Date')+1
df_reg.insert(col_pos, 'phase', df_reg.pop('phase'))

#post season
df_post['phase'] = 'post'
col_pos = df_post.columns.get_loc('Date')+1
df_post.insert(col_pos, 'phase', df_post.pop('phase'))

In [ ]:
# join the regular and post seasons dataframes together
# sort by date so its in chronological order

df = pd.concat([df_reg, df_post], ignore_index=True)
df = df.sort_values('Date').reset_index(drop=True)

In [ ]:
#change the unnamed: 6 column to home/away

df = df.rename(columns={'Unnamed: 6' : 'Home/Away'})


# convert the values in home/away to home or away
# super bowl is down as N for neutral so account for that too
# write a lambda function to do this

df['Home/Away'] = df['Home/Away'].apply(
    lambda x: 'away' if x == '@' 
              else 'home' if pd.isna(x) 
              else 'neutral' if x == 'N' 
              else x
)



<p>
So a lot of the columns have the same names due to different sub headings <br>
Like passing, rushing and recieving. <br>
Going to drop the columns I don't think I need and then rename them for easier understanding.

</p>

In [ ]:
df.head()

In [ ]:
cols_to_drop = [
    'Gcar',
    'Rk',           #Rank - not needed
    'Gtm',          #Season game number for team - we have other trackers
    'GS',           #Games Started - empty columns
    'Tgt',          #Recieving stat
    'Rec',          #recieving stat
    'Yds.3',        #recieving stat
    'Y/R',          #recieving stat
    'TD.2',         #recieving stat
    'Ctch%',        #recieving stat
    'Y/Tgt',        #recieving stat
    'Sk.1',         #defensive stat
    'Comb',         #defensive stat
    'Solo',         #defensive state
    'Ast',          #defensive stat
    'TFL',          #D state
    'QBHits',       #D stat
    'Sfty',         #team stat
    'OffSnp',       #n/a
    'Off%',         #n/a
    'DefSnp',       #n/a
    'Def%',         #n/a
    'STSnp',        #n/a
    'ST%'           #n/a
]

df = df.drop(columns=cols_to_drop)

In [ ]:
#make column titles lower case
df.columns = df.columns.str.lower()

In [ ]:
# now rename the columns
df = df.rename(columns={
    'cmp' : 'cmp_pass',
    'att' : 'pass_att',
    'cmp%' : 'cmp%_pass',
    'y/a' : 'y/a_pass',
    'ay/a' : 'ay/a_pass',
    'yds' : 'pass_yds',
    'td' : 'pass_td',
    'yds.1' : 'sk_yds',
    'att.1' : 'rush_att',
    'yds.2' : 'rush_yds',
    'td.1' : 'rush_td',
    'y/a.1' : 'y/a_rush',
    'yds.4' : 'yds_fr'    
})

In [ ]:
df.info()

In [ ]:
df[df['week']].isna()

PRACTICING A NEW COMMIOT